预测效果不好，返回来检查各feature和两个预测目标的关系

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.feature_selection import mutual_info_regression 


In [ ]:
data=pd.read_csv('lonal_entro.csv')
data.columns

In [ ]:
data.__delitem__('Unnamed: 0')
data.__delitem__('Unnamed: 0.1')

In [ ]:
data.columns

还是先看has_trip

In [ ]:
for col in data.columns:
    if col != 'trip_m6_wd_pm':
        sns.boxplot(x='has_trip', y=col, saturation=0.5, palette='pastel', data=data)
        plt.title(col)
        plt.show()

解释性全都很糟糕,或许可以认为出行量较低的od对，是否有出行是非常缺乏规律的，0还是1都是随机的结果。

has_trip可能并不是一个好编码方式，尝试多分几段

In [ ]:
data.trip_m6_wd_pm.describe()

In [ ]:
datax=data.sort_values(by='trip_m6_wd_pm')

In [ ]:
plt.style.use('bmh')
plt.figure(dpi=200)
plt.title('sort_trip')
# plt.plot(range(int(194600*0.995),194661),datax.trip_m6_wd_pm[int(194600*0.995):194661])
plt.plot(range(int(194661)),datax.trip_m6_wd_pm)
plt.savefig('pics\sort_trip.png')

数据过于稀疏。决定分成-0.1,0.5,1.5,60,5000
四段

In [ ]:
trip=data['trip_m6_wd_pm']
bins=[-0.1,0.5,1.5,60,5000]
cut_res=pd.cut(trip,bins=bins,labels=[0,1,2,4])
data['trip_intense']=cut_res

In [ ]:
for col in data.columns:
    sns.boxplot(x= 'trip_intense', y=col, saturation=0.5, palette='pastel', data=data)
    plt.title(col)
    plt.show()

发现出行量60以上的od对和其他出行强度档次的各种特征有明显区别
应该按50为界划分


In [ ]:
trip=data['trip_m6_wd_pm']
bins=[-0.1,60,5000]
cut_res=pd.cut(trip,bins=bins,labels=['a','b'])
data['trip_intense_60']=cut_res

In [ ]:
plotcol=['ori', 'dst', 'dist_km',
       'fsize_m6_wd_pm', 'trip_m6_wd_pm', 'community_m6', 'mrt_km_o',
       'entro_o', 'cycle_km_o', 'far_hdb_o', 'far_priv_o', 'far_comm_o',
       'mrt_km_d', 'entro_d', 'cycle_km_d', 'far_hdb_d', 'far_priv_d',
       'far_comm_d', 'smooth_entro_o', 'smooth_entro_d', 'ori_lat', 'ori_lon',
       'dst_lat', 'dst_lon', 'ori_lonal_pc1', 'ori_lonal_pc2',
       'dst_lonal_pc1', 'dst_lonal_pc2', 'od_lonal_pc1', 'od_lonal_pc2',
       'od_lonal_pc3', 'od_lonal_pc4', 'lonal_baseline']

In [ ]:
for col in plotcol:
    if col != 'trip_intense_60'  :
        sns.boxplot(x= 'trip_intense_60', y=col, saturation=0.5, palette='pastel', data=data)
        plt.title(col)
        plt.show()

进行数值分析，准备将trip_intense_60的切片结果作为supervised feature的基础.
或者也可能采取类似的方法，按其他标准将trip再切片，下面看看community


In [ ]:
data.groupby('trip_intense_60').describe()

In [ ]:
data.columns

In [ ]:
data["community_encoded"] = data.groupby("community_m6")["trip_m6_wd_pm"].transform("mean")


In [ ]:
data.groupby("community_m6")["trip_m6_wd_pm"].plot(legend=True)


In [ ]:
x=data.groupby("community_m6")["trip_m6_wd_pm"].agg(['mean','max','min','std','median'])
x

In [ ]:
m=data.groupby("community_m6")["trip_m6_wd_pm"]

In [ ]:
m0=m.get_group(0)
m1=m.get_group(1)
plt.boxplot([m0,m1])

In [ ]:
plt.style.use("bmh")
def make_mi_scores(X, y):
    X = X.copy()
    for colname in X.select_dtypes(["object", "category"]):
        X[colname], _ = X[colname].factorize()
    discrete_features = [pd.api.types.is_integer_dtype(t) for t in X.dtypes]
    mi_scores = mutual_info_regression(X, y, discrete_features=discrete_features, random_state=0)
    mi_scores = pd.Series(mi_scores, name="MI Scores", index=X.columns)
    mi_scores = mi_scores.sort_values(ascending=False)
    return mi_scores


def plot_mi_scores(scores):
    scores = scores.sort_values(ascending=True)
    width = np.arange(len(scores))
    ticks = list(scores.index)
    plt.barh(width, scores)
    plt.yticks(width, ticks)
    plt.title("Mutual Information Scores")
ycol='trip_m6_wd_pm'
xcol=['community_m6','community_encoded']
# y=data[ycol]
# X=data[xcol]                                
# scores=make_mi_scores(X, y)
# plot_mi_scores(scores)

更垃圾了。都是什么贵物特征，下面对经纬度pca

In [ ]:
data=pd.read_csv('lonal_entro.csv')
data.columns

In [ ]:
features_o = ["ori_lat", "ori_lon"]
features_d = ["dst_lat", "dst_lon"]
features_od=["ori_lat", "ori_lon","dst_lat", "dst_lon"]
X = data.copy()
y = X.pop('trip_m6_wd_pm')
Xo = X.loc[:, features_o]
Xd = X.loc[:, features_d]
Xod = X.loc[:, features_od]


# Standardize
Xo_scaled = (Xo - Xo.mean(axis=0)) / Xo.std(axis=0)
Xd_scaled = (Xd - Xd.mean(axis=0)) / Xd.std(axis=0)
Xod_scaled = (Xod - Xod.mean(axis=0)) / Xod.std(axis=0)

In [ ]:
from sklearn.decomposition import PCA

# Create principal components
pca = PCA()
Xo_pca = pca.fit_transform(Xo_scaled)
Xd_pca = pca.fit_transform(Xd_scaled)
Xod_pca = pca.fit_transform(Xod_scaled)

# Convert to dataframe
component_names = [f"PC{i+1}" for i in range(Xo_pca.shape[1])]
Xo_pca = pd.DataFrame(Xo_pca, columns=component_names)
Xd_pca = pd.DataFrame(Xd_pca, columns=component_names)
Xod_pca = pd.DataFrame(Xod_pca, columns=['PC1','PC2','PC3','PC4'])

data['ori_lonal_pc1']=Xo_pca['PC1']
data['ori_lonal_pc2']=Xo_pca['PC2']
data['dst_lonal_pc1']=Xd_pca['PC1']
data['dst_lonal_pc2']=Xd_pca['PC2']
data['od_lonal_pc1']=Xod_pca['PC1']
data['od_lonal_pc2']=Xod_pca['PC2']
data['od_lonal_pc3']=Xod_pca['PC3']
data['od_lonal_pc4']=Xod_pca['PC4']


In [ ]:
data.columns

In [ ]:
ycol='trip_m6_wd_pm'
xcol=['ori_lonal_pc1', 'ori_lonal_pc2', 'dst_lonal_pc1', 'dst_lonal_pc2','ori_lat', 'ori_lon', 'dst_lat', 'dst_lon','ori', 'dst','od_lonal_pc1', 'od_lonal_pc2', 'od_lonal_pc3', 'od_lonal_pc4']
y=data[ycol]
X=data[xcol]                                
scores=make_mi_scores(X, y)
plot_mi_scores(scores)

In [ ]:
pc3和pc4是不错的特征

In [ ]:
加入lonal_baseline作为基准


In [ ]:
data['lonal_baseline']=(data.ori_lon+data.ori_lat+data.dst_lon+data.dst_lat)/4

In [ ]:
ycol='trip_m6_wd_pm'
xcol=['ori_lonal_pc1', 'ori_lonal_pc2', 'dst_lonal_pc1', 'dst_lonal_pc2','ori_lat', 'ori_lon', 'dst_lat', 'dst_lon','ori', 'dst','od_lonal_pc1', 'od_lonal_pc2', 'od_lonal_pc3', 'od_lonal_pc4','lonal_baseline']
y=data[ycol]
X=data[xcol]                                
scores=make_mi_scores(X, y)
plot_mi_scores(scores)



In [ ]:
data.to_csv('lonal_entro.csv')

为什么呢,切片分组画个图看看

In [ ]:
data['od_lonal_pc4'].describe()

In [ ]:
pc1=data.sort_values('od_lonal_pc1')['od_lonal_pc1']
pc4=data.sort_values('od_lonal_pc4')['od_lonal_pc4']
x=range(194661)
# plt.plot(x,pc4,'b')
# plt.plot(x,pc1,'r')
plt.plot(data['od_lonal_pc4'],data['trip_m6_wd_pm'],'ro')


In [ ]:
plt.plot(data['od_lonal_pc3'],data['trip_m6_wd_pm'],'ro')

In [ ]:
plt.plot(data['od_lonal_pc1'],data['trip_m6_wd_pm'],'ro')

In [ ]:
第三、四主成分确实更有规律性，处理一下，。

In [ ]:

plt.plot((data['od_lonal_pc4']*data['od_lonal_pc4']),1/data['trip_m6_wd_pm'],'ro')

但回看对trip分组箱图，似乎不太对劲，是由于pc3pc4太接近0吗？
加一个baseline_0

In [ ]:
data=pd.read_csv('lonal_entro.csv')
data['baseline_0']=[0 for i in range(194661)]
data.columns

In [ ]:
ycol='trip_m6_wd_pm'
xcol=['ori', 'dst','od_lonal_pc3', 'od_lonal_pc4','lonal_baseline','baseline_0']
y=data[ycol]
X=data[xcol]                                
scores=make_mi_scores(X, y)
plot_mi_scores(scores)


看来并不是。pc4是个好feature

去forest—_test来看看对于pc4对预测效果的提高

回忆目标trip，看看能不能对目标进行重新编码

聚类

In [ ]:
n_data=pd.read_csv('new_data.csv').set_index('ori',drop=False)
n_data

In [ ]:
sg_map=pd.read_csv('sg_map.csv').set_index('id')
sg_map

In [ ]:
new_sg=pd.concat([n_data,sg_map],axis=1,join='inner')

In [ ]:
new_sg

In [ ]:
new_sg.to_csv('sg_trip_lonal.csv')

In [ ]:
from sklearn.datasets import load_iris

from sklearn.preprocessing import MinMaxScaler

from sklearn.cluster import KMeans
Xcol=['trip', "Lat", "Lon"]
X = new_sg[Xcol]
X.head()





In [ ]:

kmeans = KMeans(n_clusters=4)
X["Cluster"] = kmeans.fit_predict(X)
X["Cluster"] = X["Cluster"].astype("category")

X.head()


In [ ]:
new_sg['cluster']=X["Cluster"]

In [ ]:
new_sg.to_csv('sg_trip_lonal.csv')

In [ ]:
plt.figure(dpi=200)
sns.scatterplot(x=new_sg['Lon'], y=new_sg['Lat'],hue=new_sg['cluster'],size=0.1)

整理一下

In [ ]:
new_sg=pd.read_csv('sg_trip_lonal.csv')
new_sg.groupby('cluster')['trip'].agg(['mean','max','min','median','count'])

In [ ]:
modi=[]
for i in new_sg['cluster'].values:
    if i==0:
        modi.append(i)
    elif i==3:
        modi.append(1)
    elif i==1:
        modi.append(2)
    else:
        modi.append(3)
new_sg['cluster']=modi


In [ ]:
plt.figure(dpi=200)
sns.scatterplot(x=new_sg['Lon'], y=new_sg['Lat'],hue=new_sg['cluster'],size=0.1)

把cluster加到od里面去

In [ ]:
main_data=pd.read_csv('lonal_entro.csv')
main_data.head()
  

In [ ]:
new_sg=new_sg.set_index('ori')
new_sg

In [ ]:
ori=main_data['ori'].values
sgclus=new_sg.cluster
ori_cluster=[]
dst_cluster=[]

for i in ori:
    try:
        ori_cluster.append(sgclus[i])
    except:
        ori_cluster.append(0)

dst=main_data['dst'].values
for i in dst:
    try:
        dst_cluster.append(sgclus[i])
    except:
        dst_cluster.append(0)




In [ ]:
main_data['ori_cluster']=ori_cluster
main_data['dst_cluster']=dst_cluster
main_data.to_csv('lonal_entro.csv')


In [ ]:
ycol='trip_m6_wd_pm'
xcol=['ori', 'dst','od_lonal_pc3', 'od_lonal_pc4','lonal_baseline','ori_cluster','dst_cluster']
y=main_data[ycol]
X=main_data[xcol]                                
scores=make_mi_scores(X, y)
plot_mi_scores(scores)

In [ ]:
仅供参考？尝试回归计算

In [ ]:
main_data.columns